<a href="https://colab.research.google.com/github/Imperial-lord/sdn-controllers-load-balancing/blob/main/SDN_Load_Balancing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [20]:
import networkx as nx
import time

from math import radians, cos, sin, asin, sqrt


k = int(input("Enter the number of controllers to allocate: "))
start = time.time()
graph = nx.read_gml('Arnes.gml', label='id')

lat = nx.get_node_attributes(graph, "Latitude")
lon = nx.get_node_attributes(graph, "Longitude")

# Remove bad nodes from 'graph'
for node in list(graph.nodes):
    # removing nodes 
    # (a) with undefined latitude or longitude  
    # (b) that are isolated
    check_bad_node = lat.get(node) == None or lon.get(node) == None or nx.is_isolate(graph,node)
    if check_bad_node == True:
        graph.remove_node(node)


def dist_between_nodes(node_1, node_2):
    '''Compute distance between any 2 nodes of 'graph' using Haversine formula

    Args:
        node_1 (int): The first among the 2 graph nodes 
        node_2 (int): The second among the 2 graph nodes 

    Returns:
        dist: distanct between the two nodes
    '''
    lat_1 = radians(lat.get(node_1))
    lat_2 = radians(lat.get(node_2))
    lon_1 = radians(lon.get(node_1))
    lon_2 = radians(lon.get(node_2))

    # Applying Haversine formula
    dlon = lon_2 - lon_1
    dlat = lat_2 - lat_1
    a = sin(dlat/2)**2 + cos(lat_1)*cos(lat_2)*sin(dlon/2)**2
    c = 2*asin(sqrt(a))

    # Mean radius of Earth
    R = 6371.009
    dist = c*R

    return dist

# Assigning weights (based on distance) to the edges of the graph
for node in list(graph.nodes):
    for neighbor in list(graph.neighbors(node)):
        graph[node][neighbor]['weight'] = dist_between_nodes(node, neighbor)

# Removing edges with no weights assigned
for edge in list(graph.edges):
    if 'weight' not in graph.edges[edge] == False:
        graph.remove_edge(edge[0],edge[1])

print(nx.info(graph))


Enter the number of controllers to allocate: 20
Graph with 34 nodes and 46 edges
